In [1]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [9]:
device = torch.device('cpu')

In [31]:
size = 10

In [32]:
def createModel():
    d_out = 1


    first_layer = 4
    model = rtdl.MLP.make_baseline(
        d_in=size,
    #     d_layers=[first_layer, 256, 128],
        d_layers=[first_layer, 8, first_layer],
        dropout=0.1,
        d_out=d_out,
        # seed=42
    )
    lr = 0.001
    weight_decay = 0.0

   

    model.to(device)
    optimizer = (
        model.make_default_optimizer()
        if isinstance(model, rtdl.FTTransformer)
    #     else torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        else torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    )
    loss_fn = (
        F.mse_loss
    )
    return model, optimizer, loss_fn


In [43]:
def apply_model(x_num, x_cat=None, model=None):
    if isinstance(model, rtdl.FTTransformer):
        return model(x_num, x_cat)
    elif isinstance(model, (rtdl.MLP, rtdl.ResNet)):
        assert x_cat is None
        return model(x_num)
    else:
        raise NotImplementedError(
            f'Looks like you are using a custom model: {type(model)}.'
            ' Then you have to implement this branch first.'
        )

In [44]:
@torch.no_grad()
def evaluate(part, model):
    model.eval()
    prediction = []
    for batch in zero.iter_batches(X[part], 1024):
        prediction.append(apply_model(batch,model=model))
    prediction = torch.cat(prediction).squeeze(1).cpu().numpy()
    target = y[part].cpu().numpy()

    if task_type == 'binclass':
        prediction = np.round(scipy.special.expit(prediction))
        score = sklearn.metrics.accuracy_score(target, prediction)
    elif task_type == 'multiclass':
        prediction = prediction.argmax(1)
        score = sklearn.metrics.accuracy_score(target, prediction)
    else:
        assert task_type == 'regression'
        score = sklearn.metrics.mean_squared_error(target, prediction) ** 0.5 * y_std
    return score

In [45]:
model, optimizer, loss_fn = createModel()

In [46]:
params = list(model.named_parameters())

In [52]:
params[0][1].grad

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.2641, 0.2641, 0.2641, 0.2641, 0.2641, 0.2641, 0.2641, 0.2641, 0.2641,
         0.2641],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])

In [51]:
x = torch.ones((4,size))
y = torch.ones((1,1))

loss = loss_fn(apply_model(x,model=model).squeeze(1), y)
loss.backward()

C:\Users\PaulPeseux\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  after removing the cwd from sys.path.


In [55]:
layer = params[0][1]

In [56]:
# params[0][1].grad

In [57]:
do = nn.Dropout(p=0.2)

In [58]:
do(layer)

tensor([[-0.0606, -0.0099,  0.3611, -0.2599, -0.3535, -0.1250,  0.1934, -0.1537,
          0.0000, -0.2951],
        [ 0.3844,  0.1660,  0.2433, -0.2555,  0.0000, -0.0711, -0.0000, -0.0000,
          0.0985,  0.2644],
        [-0.0000,  0.0000, -0.0000, -0.1603, -0.2079, -0.1869, -0.0743,  0.2212,
         -0.2119, -0.3578],
        [-0.3835,  0.0000,  0.3686, -0.2478, -0.0810,  0.2175,  0.0000, -0.2495,
         -0.1506,  0.1743]], grad_fn=<MulBackward0>)